# Fenotipagem de tomateiro resistente à requeima

## Introdução

A fenotipagem é o processo de avaliação de características mensuráveis, como peso, formato dos frutos e estruturas vegetais. O uso de imagens capturadas em diferentes regiões do espectro eletromagnético permite estimar medidas essenciais, como altura, largura e número de folhas, para monitorar o crescimento vegetativo.

A requeima, ou *mela*, é uma doença causada pelo oomiceto *Phytophthora infestans*, que afeta culturas de tomate. Embora possa causar a perda total da cultura, existem cultivares resistentes à doença. Experimentos com diferentes variedades podem avaliar a severidade da requeima em diversas cultivares, utilizando imagens de drone.

De acordo com o Código Internacional de Nomenclatura de Plantas Cultivadas (**ICNCP**), uma cultivar é definida como um "*conjunto de plantas selecionado por atributos específicos ou combinação de atributos*". Dada a variedade de cultivares disponíveis, um experimento foi conduzido com o uso de imagens de drone, que foram processadas e analisadas com ferramentas de *machine learning* para economizar tempo e recursos.

Uma câmera multiespectral **MicaSense**, equipada com cinco bandas espectrais (RGB, infravermelho próximo (NIR) e Red Edge), foi utilizada para capturar as imagens. Os índices de vegetação, calculados a partir dessas bandas espectrais, foram processados e resultaram no *dataset* utilizado para prever a severidade da requeima.


## Análise preditiva

### Importamos as bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor 
from sklearn.tree import DecisionTreeRegressor 
from sklearn.svm import SVR 
from sklearn.model_selection import cross_val_score 
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score 
from sklearn.feature_selection import RFE 

### Carregamos o dataset

In [ ]:
df = pd.read_csv('dataset_sintetico.csv')

X = df.drop(['id', 'Severidade'], axis = 1)
Y = df['Severidade']

### Extraímos uma amostra dos dados

In [ ]:
df.head()

### Funções para ajuste dos modelos

In [ ]:
def knn_regressor():
    neigh_score = []
    for i in range(1, 30):
        knn = KNeighborsRegressor(n_neighbors=i)
        knn.fit(X_train, Y_train)
        pred = knn.predict(X_test)
        score = (mean_squared_error(Y_test, pred)**0.5)
        neigh_score.append((i, score))
    
    # Seleciona o k com o menor erro
    k = min(neigh_score, key=lambda x: x[1])[0]
    knn = KNeighborsRegressor(n_neighbors=k)
    return knn

In [ ]:
def svr_regressor():
    svr_scores = []
    for c in [0.01, 0.1, 1, 10, 100]:
        svr = SVR(C=c)
        svr.fit(X_train, Y_train)
        pred = svr.predict(X_test)
        score = mean_squared_error(Y_test, pred)
        svr_scores.append((c, score))
    
    # Seleciona o C com o menor erro
    best_c = min(svr_scores, key=lambda x: x[1])[0]
    svr = SVR(C=best_c)
    return svr

In [ ]:
def decision_tree_regressor():
    tree_scores = []
    for depth in range(1, 30):
        tree = DecisionTreeRegressor(max_depth=depth)
        tree.fit(X_train, Y_train)
        pred = tree.predict(X_test)
        score = mean_squared_error(Y_test, pred)
        tree_scores.append((depth, score))
    
    # Seleciona a profundidade com o menor erro
    best_depth = min(tree_scores, key=lambda x: x[1])[0]
    tree = DecisionTreeRegressor(max_depth=best_depth)
    return tree

In [ ]:
def random_forest_regressor():
    forest_scores = []
    for n in range(10, 200, 10):
        forest = RandomForestRegressor(n_estimators=n)
        forest.fit(X_train, Y_train)
        pred = forest.predict(X_test)
        score = mean_squared_error(Y_test, pred)
        forest_scores.append((n, score))
    
    # Seleciona o número de árvores com o menor erro
    best_n = min(forest_scores, key=lambda x: x[1])[0]
    forest = RandomForestRegressor(n_estimators=best_n)
    return forest

In [ ]:
def hist_gradient_boosting_regressor():
    hgb_scores = []
    for l_rate in [0.01, 0.1, 0.2, 0.3, 0.4, 0.5]:
        hgb = HistGradientBoostingRegressor(learning_rate=l_rate)
        hgb.fit(X_train, Y_train)
        pred = hgb.predict(X_test)
        score = mean_squared_error(Y_test, pred)
        hgb_scores.append((l_rate, score))
    
    # Seleciona a taxa de aprendizado com o menor erro
    best_rate = min(hgb_scores, key=lambda x: x[1])[0]
    hgb = HistGradientBoostingRegressor(learning_rate=best_rate)
    return hgb

In [ ]:
# modelos = {"Regressão Linear": LinearRegression, "KNN": KNeighborsRegressor, "SVR": svr_regressor, "Decision Tree": DecisionTreeRegressor, "Random Forest": RandomForestRegressor, "HGB": HistGradientBoostingRegressor }
tunning = {"Regressão Linear": LinearRegression, "KNN": knn_regressor, "SVR": svr_regressor, "Decision Tree": decision_tree_regressor, "Random Forest": random_forest_regressor, "HGB": hist_gradient_boosting_regressor }

### Separamos os dados de treinamento e teste

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

### Ajuste dos modelos sem pre-processamento ou seleção de "features"

In [ ]:
# modelo = LinearRegression()
for name in tunning:
    model = tunning[name]()
    model.fit(X_train, Y_train).predict(X_train)
    
    print(name)
    score = cross_val_score(model, X_train, Y_train, cv = 10, scoring='r2')
    print(np.mean(score))
    print("=======================================")
    
    # plt.plot(fpr, tpr, label='%s (area = %0.2f)' % (name, roc_auc))
    # print(X.shape)()


### Função para seleção de "features"

In [ ]:

def best_k(X, Y):
    err = float('inf')
    best_val = 0
    
    for i in range(1, X.shape[1] + 1):
        model = LinearRegression()
        selector = RFE(model, n_features_to_select=i)
        X_new = selector.fit_transform(X, Y)
        
        scores = cross_val_score(model, X_new, Y, cv=10, scoring='neg_mean_squared_error')
        mse = -scores.mean()
        
        if mse < err:
            err = mse
            best_val = i
          
    return best_val


### Ajustes dos modelos com pré-processamento e seleção de "features"

In [ ]:
# Padronizando os dados
X = StandardScaler().fit_transform(X)

# Seleção de features usando a função best_k
K = best_k(X, Y)
selector = RFE(LinearRegression(), n_features_to_select=K)
X_new = selector.fit_transform(X, Y)

# Divisão os dados em treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X_new, Y, train_size=0.7)

# Treinamento e validação cruzada para seleção do melhor modelo
best_score = -float('inf')
best_model_name = None
best_model = None

for name in tunning:
    model = tunning[name]()
    scores = cross_val_score(model, X_train, Y_train, cv=10, scoring='r2')
    mean_score = scores.mean()
    
    print(f'Modelo: {name}, Média R2: {mean_score}')
    
    if mean_score > best_score:
        best_score = mean_score
        best_model_name = name
        best_model = model

print("=======================================")
print(f"Melhor modelo: {best_model_name} apresentou R2: {best_score}")


# Avaliação do melhor modelo no conjunto de teste

In [ ]:
best_model.fit(X_train, Y_train)
y_pred = best_model.predict(X_test)

r2 = r2_score(Y_test, y_pred)
mse = (mean_squared_error(Y_test, y_pred))**0.5
mae = mean_absolute_error(Y_test, y_pred)

print("Avaliação do modelo final com os dados de teste")
print('r2:', r2)
print('mse:', mse)
print('mae:', mae)
